# 语言模型

目标：给定一个句子，模型输出这句话出现的概率是多少，这样的话可以做很多任务，比如给定一句话进行填词，文本的生成等。

标准定义：给定语言序列$w_1,w_2,...,w_n$，语言模型就是计算该序列的概率，由链式法则：

$P(w_1,w_2,...,w_n)=P(w_1)P(w_2|w_1)\cdots P(w_n|w_1,...,w_{n-1})$

统计语言模型中，采用极大似然估计来计算每个词出现的条件概率，但对于任意长的自然语言语句，这个方法可能行不通，于是后来就引入马尔科夫假设，即当前次出现的概率只依赖于前n-1个词，于是得到如下公式：

$P(w_i|w_1,w_2,...,w_{i-1})=P(w_i|w_{i-n+1},...,w_{i-1})$

于是最终结论为：

$P(w_1,w_2,...,w_m)=\prod^m_{i=1}P(w_i|w_1,w_2,...,w_{i-1})\approx \prod^m_{i=1}P(w_i|w_{i-n+1},...,w_{i-1})$

本次任务的目的就是通过循环神经网络模型来计算这个概率，相比较单纯的前馈神经网络，隐状态的传递性使得RNN语言模型原则上可以捕捉前向序列的所有信息（虽然可能比较弱）。

通过在整个训练集上优化交叉熵来训练模型，使得网络能够尽可能建模出自然语言序列与后续词之间的内在联系。

本文主要实现这样一个模型：
<img style="float: center;" src="images/7.png" width="70%">

模型的输入是一个句子，模型的输出也是一个句子。
- 比如"I like China"

在第一个时间步的时候输入"I"，然后通过一个embedding层得到"I"的词向量表示，通过LSTM网络得到当前时间步的稳态值，经过一个全连接网络得到"like"，然后"like"作为第二个时间步的输入，得到第二个时间步的输出"China"，这样一步步地进行传递，进行模型训练。

当下一次输出"I"的时候，模型可能就会继续输出"like China"，这样就能得到我们想要的句子，使得模型的输出更加看起来像"人话"，相当于$P(I like China)>P(like China I)$

# 实现一个RNN语言模型

依然用之前的text8，里面分成了训练集，交叉验证集和测试集，每个数据集里是一段文字，每个单词之间用空格进行隔开。

处理的方式依然是拿到这些数据集，然后构建词典，数据迭代器，index_to_word，word_to_index这些东西，不过这次用torchtext这个工具包完成任务。

其次就是语言模型的定义，这里会搭建一个模型，然后进行模型的训练，使用的损失函数为交叉熵损失，因为每一个时间步其实在做一个多分类的问题，所以这个损失函数刚刚是解决这样的一个任务，训练过程就是前向传播，计算损失，反向传播，梯度更新，梯度清零，模型保存等。

这里也会涉及语言模型的一些训练细节，比如隐藏状态的batch之间的传递，梯度修剪，动态更新学习率等。

## 导入包

In [25]:
import torchtext
from torchtext.vocab import Vectors
import torch
import torch.nn as nn
import numpy as np
import random

USE_CUDA = torch.cuda.is_available()

random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
if USE_CUDA:
    torch.cuda.manual_seed(1)
    
    
## 定义常用的参数
BATCH_SIZE = 32
EMBEDDING_SIZE = 100
HIDDEN_SIZE = 64
MAX_VOCAB_SIZE = 30000
NET_LAYER_SIZE = 2

device = torch.device('cuda' if USE_CUDA else 'cpu')

## 构建词典和迭代器

In [19]:
# Field决定了数据如何被处理，这个例子中先转换成小写
TEXT = torchtext.legacy.data.Field(lower=True)
train, val, test = torchtext.legacy.datasets.LanguageModelingDataset.splits(path='./data/', 
                    train='text8.train', validation='text8.dev', test='text8.test', text_field=TEXT)
TEXT.build_vocab(train, max_size=MAX_VOCAB_SIZE)
len(TEXT.vocab)  # 那两个是两个外界符号   30002

30002

构建词典基本上是用torchtext包完成：
- torchtext的一个重要概念是Field，它决定数据会被如何处理。使用TEXT的Field来处理文本数据，lower=True将所有的单词都变成小写。
- torchtext提供LanguageModelingDataset类来帮助处理语言模型数据集。
- build_vocab根据提供的训练数据集来创建最高频单词的单词表，max_size帮助限顶单词总量

根据上一步，不仅构造好了词典，同时也制作好了index_to_word和word_to_index两种映射

In [20]:
TEXT.vocab.itos[:10]   # index to string   index to word
TEXT.vocab.stoi['apple']    # word to index 

1259

获取训练集，验证集和测试集的iter，BPTTIterator可以连续地得到连贯地句子

In [21]:
VOCAB_SIZE = len(TEXT.vocab)

train_iter, val_iter, test_iter = torchtext.legacy.data.BPTTIterator.splits(
    (train, val, test), batch_size=BATCH_SIZE, device=device, 
    bptt_len=50, repeat=False, shuffle=True)
# 这里是50个句子为一句，下面的text和target都是50*32，50代表单词的个数，32代表batch的大小
next(iter(train_iter))


[torchtext.legacy.data.batch.Batch of size 32]
	[.text]:[torch.cuda.LongTensor of size 50x32 (GPU 0)]
	[.target]:[torch.cuda.LongTensor of size 50x32 (GPU 0)]

查看数据

In [22]:
it = iter(train_iter)
batch = next(it)
print(" ".join([TEXT.vocab.itos[i] for i in batch.text[:, 1].data]))
print(" ".join([TEXT.vocab.itos[i] for i in batch.target[:, 1].data]))

combine in pairs and then group into trios of pairs which are the smallest visible units of matter this parallels with the structure of modern atomic theory in which pairs or triplets of supposedly fundamental quarks combine to create most typical forms of matter they had also suggested the possibility
in pairs and then group into trios of pairs which are the smallest visible units of matter this parallels with the structure of modern atomic theory in which pairs or triplets of supposedly fundamental quarks combine to create most typical forms of matter they had also suggested the possibility of


模型输入是一串数字，输出也是一串数字，它们之间相差一个位置。

语言模型的目标是根据之前的单词预测下一个单词，如果不明显可以多看几个batch：

In [23]:
for k in range(5):
    batch = next(it)
    print(k)
    print(" ".join([TEXT.vocab.itos[i] for i in batch.text[:, 2].data]))
    print()
    print(" ".join([TEXT.vocab.itos[i] for i in batch.target[:, 2].data]))

0
reject that the relationship goes beyond contact i e mutual borrowing of words between japanese and ainu in fact no attempt to show a relationship with ainu to any other language has gained wide acceptance and ainu is currently considered to be a language isolate culture traditional ainu culture is

that the relationship goes beyond contact i e mutual borrowing of words between japanese and ainu in fact no attempt to show a relationship with ainu to any other language has gained wide acceptance and ainu is currently considered to be a language isolate culture traditional ainu culture is quite
1
quite different from japanese culture never <unk> after a certain age the men had full <unk> and <unk> men and women alike cut their hair level with the shoulders at the sides of the head but trimmed it <unk> behind the women <unk> their mouths arms <unk> and sometimes their

different from japanese culture never <unk> after a certain age the men had full <unk> and <unk> men and women alike cu

可以发现每个batch之间的句子是连起来的，下一个batch的开头正好是上一个batch的结尾。
<img style="float: center;" src="images/8.png" width="70%">

训练时，会采用一些技巧进行batch与batch之间隐藏状态的传递。

注意，如果不是训练语言模型，一般batch之间是没有啥关系的。

因此这个技巧一般只适合语言模型。

## 定义模型

继承nn.Module，然后初始化函数，前向传播。

以上三个是必备的，其他根据需要自定义。

In [26]:
class RNNModel(nn.Module):
    """一个简单的循环神经网络, 
       主要包括词嵌入层，
       一个循环神经网络，
       一个线性层，
       一个Dropout层
    """
    
    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5):
        """
        rnn_type: 表示rnn的类型，有RNN，LSTM，GRU
        ntoken: 词典内的单词个数
        ninp: 这个是输入维度， 也是embedding维度
        nhid: 这个是神经网络隐藏单元的个数
        nlayers: 这个是神经网络的层数
        """
        super(RNNModel, self).__init__()
        
        # dropout层要放在最前面
        self.drop = nn.Dropout(dropout)
        self.embed = nn.Embedding(ntoken, ninp)
        if rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        else:
            self.rnn = RNN(ninp, nhid, nlayers, nonlinearity='relu', dropout=dropout)
        
        self.linear = nn.Linear(nhid, ntoken)
        
        self.init_weights()
        
        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers
    
    # 权重初始化
    def init_weights(self):
        initrange = 0.1
        self.embed.weight.data.uniform_(-initrange, initrange)   # 下划线代表原位操作
        self.linear.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.zero_()
        
    # forward
    def forward(self, input, hidden):
        """
         这个过程就是先embedding，然后经过LSTM，然后通过一个线性层转换输出单词表
         input: 指的是当前时间步的输入
         hidden：指的是前一隐藏状态
        """
        # input: (seq_len, batch)     
        # torch的LSTM的默认输入是第一个维度是seq_len，第二个维度是batch
        # 那里指定batch_first=True
        embed = self.drop(self.embed(input))      # (seq_len, batch, embedding_size)
        output, hidden = self.rnn(embed, hidden)  
        # output: (seq_len, batch, hidden_size)   
        # hidden: [(1, batch_size, hidden_size), (1, batch_size, hidden_size)]  一个h， 一个c 对于LSTM
        output = self.drop(output)
        linear = self.linear(output.view(-1, output.size(2)))   # [seq_len*batch, vocab_size]
        return linear.view(output.size(0), output.size(1), linear.size(1)), hidden   
       # 这个地方之所以要把hidden也返回， 是因为hidden表示batch的最后一个时间步的隐藏状态信息， 而通过之前我们发现， 语言模型里面
        # batch与batch之间是相联系的， 上一个batch的最后一个单词正好是下一个batch的第一个单词， 那么就像最后一个单词的batch保存下来
        # 供下面的batch使用， 使得这个hidden在batch之间进行传递
    
    
    def init_hidden(self, bsz, requires_grad=True):    
        # 初始化隐藏层状态有个技巧了  隐藏层状态的大小是(nlayers, batch_size, hidden_size), 但是我们并不知道此时的参数是在cuda还是
        # 在CPU上， 所以下面一个代码是获得某一次batch时的权重参数， 通过这个我们初始化hidden， 这样就可以保证类型一样了
        weight = next(self.parameters())   # 获取某个iter时的参数
        if self.rnn_type == 'LSTM':       # 因为LSTM的时候是有c和h的
            return (weight.new_zeros((self.nlayers, bsz, self.nhid), requires_grad=requires_grad),  # 创建0张量， 且和weight是同一类型
                   weight.new_zeros((self.nlayers, bsz, self.nhid), requires_grad=requires_grad))
        else:
            return weight.new_zeros((self.nlayers, bsz, self.nhid), requires_grad=requires_grad)

建立模型，参数包括词典中单词的个数，embedding的维度，神经网络隐藏单元的个数，网络的层数，这些在具体运算中确定了参数的维度信息。

在初始化隐藏状态信息的时候用到了一个技巧，就是当不知道参数在GPU还是CPU上，所以那个地方是获取了目前模型的参数，通过这个参数建立张量，这样就保证了新建立的张量和模型的参数类型一致，这样就解决了是不是GPU或者CPU的问题。

In [27]:
# 初始化一个模型
model = RNNModel("LSTM", VOCAB_SIZE, EMBEDDING_SIZE, HIDDEN_SIZE, NET_LAYER_SIZE, dropout=0.5)
if USE_CUDA:
    model = model.cuda()

## 模型训练与评估

定义一个函数，获取隐藏层状态。

因为通过前面的数据发现，每一个batch之间是有联系的，上一个batch的最后一句正好是下一个batch的第一句话，这样其实batch之间的隐藏状态也往下传递。

但是这样面临的一个问题就是如果句子非常长，很容易梯度消失，所以需要只获取隐藏状态的值，而不需要整个计算图。

这个函数就是单纯地拿出隐藏状态地值，而脱离了计算图，其涉及到PyTorch地hook机制。

In [36]:
# 需要先定义一个function，帮助我们把一个hidden state和计算图之前的历史分离
def repackage_hidden(h):
    if isinstance(h, torch.Tensor):
        return h.detach()    # 这是钩子函数里面的一个机制， 从计算图里面摘下来
    else:
        return tuple(repackage_hidden(v) for v in h)  # 这个应该是h和c同在的时候， 就需要分开来进行摘下

定义模型的损失函数和优化器

In [37]:
loss_fn = nn.CrossEntropyLoss()
lr = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.5)

训练之前，定义一个验证函数，用于保存效果比较好地模型

In [38]:
# 首先定义模型评估的代码，模型评估的代码和模型的训练逻辑基本相同
# 唯一的区别就是不需要反向传播，只需要正向传播
def evaluate(model, data):
    model.eval()   #  这个声明一下模型的状态
    total_loss = 0.
    it = iter(data)
    total_count = 0.
    with torch.no_grad():   # 注意，验证的时候就不需要求梯度了
        hidden = model.init_hidden(BATCH_SIZE, requires_grad=False) # 初始化隐藏状态
        for i, batch in enumerate(it):
            data, target = batch.text, batch.target       # 获取X, Y
            if USE_CUDA:
                data, target = data.cuda(), target.cuda()
            hidden = repackage_hidden(hidden)   # 只要隐藏状态
            with torch.no_grad():
                output, hidden = model(data, hidden)      # 前向传播

            loss = loss_fn(output.view(-1, VOCAB_SIZE), target.view(-1))   # 计算损失
            total_count += np.multiply(*data.size())    # np.multiply后面这个其实是data的两个shape相乘， *表示可变参数,算的是单词总个数
            total_loss += loss.item() * np.multiply(*data.size())
    
    loss = total_loss / total_count
    model.train() # 验证完了之后，别忘了恢复模型的训练状态
    return loss

模型训练思路：
- 模型训练需要若干个epoch
- 每个epoch，先获取数据的迭代器，然后分成多个batch
- 每个batch的输入和输出都包装成cuda tensor
- 前向传播，通过输入的句子预测每个单词的下一个单词
- 用模型的预测和正确的下一个单词计算交叉熵损失
- 清空模型当前的梯度
- 反向传播
- 梯度裁剪，防止梯度爆炸
- 更新模型参数
- 每隔一定的iteration输出模型在当前iteration的loss，在验证集上做模型的评估

In [39]:
GRAD_CLIP = 1.
NUM_EPOCHS = 2

val_losses = []
for epoch in range(NUM_EPOCHS):
    model.train()
    it = iter(train_iter)      # 获得数据的迭代器
    hidden = model.init_hidden(BATCH_SIZE)   # 初始化hidden为0
    for i, batch in enumerate(it):
        data, target = batch.text, batch.target
        if USE_CUDA:
            data, target = data.cuda(), target.cuda()     # 包装成cuda
        hidden = repackage_hidden(hidden)   # 这一步是为了把隐藏状态从计算图中取下来
        
        #梯度清零
        model.zero_grad()
        # forward pass 
        output, hidden = model(data, hidden)
        # loss
        #print(output.view(-1, VOCAB_SIZE), target.view(-1))  # 前者是一个[1600, 50002] 后者是一个1600
        # 我猜是这么算的，后者里面是正确单词的位置，那么对应前面每一行相应的那个位置的数， 
        # 那么就用e^loc[target[i]] / e^out_put[i][j] j从1到50002 然后-log， 这样就会得出每一行的结果， 相当于依据
        # 后面的target做了一个softmax， 然后取了-log， 最后再把所有行的那个结果取了一个平均

        # 前者是一个(1600,30002)的矩阵，后者是一个(1600，)的向量
        # 具体计算的时候先通过后面的target里的位置获取每一行相应位置的数值
        # 然后通过softmax得到每一行里目标单词的概率，然后log一下取负号就得到每一行的交叉熵损失
        # 最后再求个平均得到了最终的loss
        loss = loss_fn(output.view(-1, VOCAB_SIZE), target.view(-1))   
        # backward 
        loss.backward()
        # 梯度修剪
        torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
        optimizer.step()
        
        if i % 1000 == 0:
            print("epoch", epoch, "iter", i, "loss", loss.item())
        
        # 模型评估和保存
        if i % 10000 == 0:
            val_loss = evaluate(model, val_iter)
            # 模型保存时保存当前验证最小的模型，保存参数字典的方式进行保存
            if len(val_losses) == 0 or val_loss < min(val_losses):
                print('best model, val loss: ', val_loss)
                torch.save(model.state_dict(), 'lm-best.th')
            else:
                # 学习率衰减
                scheduler.step()
                optimizer = torch.optim.Adam(model.parameters(), lr=lr)
            val_losses.append(val_loss)

epoch 0 iter 0 loss 10.3095064163208
best model, val loss:  10.30363225273546
epoch 0 iter 1000 loss 6.710871696472168
epoch 0 iter 2000 loss 6.775953769683838
epoch 0 iter 3000 loss 6.623823165893555
epoch 0 iter 4000 loss 6.368831634521484
epoch 0 iter 5000 loss 6.605158805847168
epoch 0 iter 6000 loss 6.412139892578125
epoch 0 iter 7000 loss 6.299011707305908
epoch 0 iter 8000 loss 6.321042537689209
epoch 0 iter 9000 loss 6.231029033660889
epoch 1 iter 0 loss 6.3042311668396
best model, val loss:  6.021427464723173
epoch 1 iter 1000 loss 6.043297290802002
epoch 1 iter 2000 loss 6.233555316925049
epoch 1 iter 3000 loss 6.2158284187316895
epoch 1 iter 4000 loss 5.957834243774414
epoch 1 iter 5000 loss 6.3020782470703125
epoch 1 iter 6000 loss 6.099340915679932
epoch 1 iter 7000 loss 6.138329029083252
epoch 1 iter 8000 loss 6.104619026184082
epoch 1 iter 9000 loss 6.057351112365723


导入模型

In [40]:
# 导入最好的模型
best_model = RNNModel("LSTM", VOCAB_SIZE, EMBEDDING_SIZE, HIDDEN_SIZE, NET_LAYER_SIZE, dropout=0.5)
if USE_CUDA:
    best_model = best_model.cuda()
best_model.load_state_dict(torch.load('lm-best.th'))

<All keys matched successfully>

进行测试

In [41]:
# 使用最好的模型在valid数据上计算perpleity
val_loss = evaluate(best_model, val_iter)
print('perplexity', np.exp(val_loss))

# 使用最好的模型在test数据上计算perplexity
test_loss = evaluate(best_model, test_iter)
print('perplexity', np.exp(test_loss))

perplexity 412.166529180177
perplexity 412.166529180177


使用训练好的模型生成一些句子

In [42]:
# 使用训练好的模型来生成一些句子
hidden = best_model.init_hidden(1)   # batch_size大小为1
input = torch.randint(VOCAB_SIZE, (1, 1), dtype=torch.long).to(device)   # 在0-30002中随机的产生一个单词， 作为第一时间步的输入
words = []
for i in range(100):
    output, hidden = best_model(input, hidden)   # 前向传播
    word_weights = output.squeeze().exp().cpu()  # 计算权重
    word_idx = torch.multinomial(word_weights, 1)[0]   # 根据权重进行采样
    input.fill_(word_idx)
    word = TEXT.vocab.itos[word_idx]     # 得到单词
    words.append(word)
print(" ".join(words))   # 单词拼接成句子

totally hampshire sparked its ordinary beer lawful christians is and grew all one nine zero pc theft on <unk> in their a past while or assuming to the ideology food the core to be busy each element beginning while six zero watch <unk> a environment of lightning in poem can have vowel consumption by multiplications a potential is to organic own philosophical though day however travelling has a new reasoning cable eclipses revenue makes when ai the root was computed by <unk> recognizes bright pulsar in language the other engines quit chooses offline or a unrestricted district has still t
